In [4]:
import Torch

In [5]:
batchSize = 16
windowSize = 7
numSample = 1000


# DATA

In [14]:
import qualified Torch.Functional.Internal as FI
import Torch

type Dataloader = [(Tensor, Tensor)]
type MyDataset = [(Tensor, Tensor)]

createInput :: Tensor -> Int -> [Tensor]
createInput tensor winSize =
    [ Torch.squeezeAll $ FI.slice tensor 0 i (i + winSize) 1 | i <- [0 .. Torch.size 0 tensor - winSize] ]

-- Create target values (next steps after window)
-- Output shape: [1, featureSize]
createTarget :: Tensor -> Int -> [Tensor]
createTarget tensor winSize = [ FI.slice tensor 0 (i + winSize) (i + winSize + 1) 1 | i <- [0 .. Torch.size 0 tensor - winSize - 1] ]

-- Create dataset of (input, target) pairs
createDataset :: Tensor -> Int -> MyDataset
createDataset tensor winSize = zip (createInput tensor winSize) (createTarget tensor winSize)

-- Helper: Split a list into chunks of given size
chunksOf :: Int -> [a] -> [[a]]
chunksOf _ [] = []
chunksOf n xs = let (chunk, rest) = splitAt n xs in chunk : chunksOf n rest

-- Helper: Stack a list of tensors along a new 0 axis
-- Input: List of tensors, each potentially with shape [S1, S2, ...]
-- Output: Tensor with shape [ListLength, S1, S2, ...]
stackTensors :: [Tensor] -> Tensor
stackTensors ts = Torch.stack (Dim 0) ts

-- Create batched dataloader
-- Input batch shape: [batchSize, windowSize, featureSize]
-- Target batch shape: [batchSize, featureSize] (after squeezing dim 1)
createDataloader :: MyDataset -> Int -> Dataloader
createDataloader dataset batchSize =
    map processBatch (filter (\chunk -> length chunk == batchSize) (chunksOf batchSize dataset))
    where
        processBatch batch =
            let inputs = map fst batch  -- List of tensors [windowSize, featureSize]
                targets = map snd batch -- List of tensors [1, featureSize]
                inputBatch = stackTensors inputs -- Tensor [batchSize, windowSize, featureSize]
                -- Stack targets to [batchSize, 1, featureSize], then squeeze to [batchSize, featureSize]
                targetBatch = Torch.squeezeDim 1 (stackTensors targets)
            in (inputBatch, targetBatch)


Line 29: Eta reduce
Found:
stackTensors ts = Torch.stack (Dim 0) ts
Why not:
stackTensors = Torch.stack (Dim 0)

In [16]:
v <- randIO' [numSample,1]

In [17]:
x = createInput v windowSize
shape $ head x

[7]

In [18]:
dataset = createDataset v windowSize

In [19]:
dataloader = createDataloader dataset batchSize
putStrLn $ "Dataloader Size : " ++ show ( length dataloader)
putStrLn $ "Size of one Item: (" ++ show (shape (fst (head dataloader))) ++ "," ++ show (shape (snd (head dataloader))) ++ ")"

Dataloader Size : 62

Size of one Item: ([16,7],[16,1])

# GRU

In [20]:
import Torch.NN.Recurrent.Cell.GRU 

-- data GRUModel = GRUModel
--   { gruLayer :: GRU
--   , linearOut :: Linear
--   } deriving (Generic, Show)




model <- sample $ GRUSpec{inputSize = 7, hiddenSize = 10}
hidden_state <- randIO' [batchSize, 10]


:t model
shape hidden_state

model :: GRUCell

[16,10]

In [22]:

(x,y) = head dataloader
shape x




[16,7]